# Train YOLO object detector with Turi Create

In [1]:
import os, sys, math
import pandas as pd
import turicreate as tc

Helper code for loading the CSV file and combining it with an SFrame. We only keep the images that we have annotations for.

In [2]:
def load_images_with_annotations(images_dir, annotations_file):
    # Load the images into a Turi SFrame.
    data = tc.image_analysis.load_images(images_dir, with_path=True)
    
    # Load the annotations CSV file into a Pandas dataframe.
    csv = pd.read_csv(annotations_file)

    # Loop through all the images and match these to the annotations from the
    # CSV file, if annotations are available for the image.
    all_annotations = []
    for i, item in enumerate(data):
        # Grab image info from the SFrame.
        img_path = item["path"]
        img_width = item["image"].width
        img_height = item["image"].height

        # Find the corresponding row(s) in the CSV's dataframe.
        image_id = os.path.basename(img_path)[:-4]
        rows = csv[csv["image_id"] == image_id]

        # Turi expects a list for every image that contains a dictionary for
        # every bounding box that we have an annotation for.
        img_annotations = []
        # The CSV file stores the coordinate as numbers between 0 and 1,
        # but Turi wants pixel coordinates in the image.
        
        # A bounding box in Turi is given by a center coordinate and the
        # width and height, we have them as the four corners of the box.

            
            # img_annotations.append({"coordinates": {"height": height, 
            #                                         "width": width, 
            #                                         "x": x, 
            #                                         "y": y}, 
            #                         "label": class_name})

        # If there were no annotations for this image, then append a None
        # so that we can filter out those images from the SFrame.
        if len(img_annotations) > 0:
            all_annotations.append(img_annotations)
        else:
            all_annotations.append(None)

    data["annotations"] = tc.SArray(data=all_annotations, dtype=list)
    return data.dropna()

In [3]:
data_dir = "snacks"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")

In [4]:
train_data = load_images_with_annotations(train_dir, data_dir + "/annotations-train.csv")

In [5]:
len(train_data)

4265

In [6]:
train_data.head()

path,image,annotations
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 341 Width: 256,"[{'coordinates':{'height': 95, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 341,"[{'coordinates':{'height': 122, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 384,"[{'coordinates':{'height': 88, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 341,"[{'coordinates':{'height': 150, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 341,"[{'coordinates':{'height': 100, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 382,"[{'coordinates':{'height': 37, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 256,"[{'coordinates':{'height': 53, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 256,"[{'coordinates':{'height': 177, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 446,"[{'coordinates':{'height': 59, 'width': ..."
/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machin ...,Height: 256 Width: 341,"[{'coordinates':{'height': 61, 'width': ..."


In [7]:
train_data[0]

{'path': '/Users/ParagonLight/OneDrive/iOS智能开发/资料/Machine_Learning_by_Tutorials_v2.4.0/10-yolo/final/snacks/train/apple/007a0bec00a90a66.jpg',
 'image': Height: 341px
 Width: 256px
 Channels: 3,
 'annotations': [{'coordinates': {'height': 95,
    'width': 112,
    'x': 73,
    'y': 113},
   'label': 'apple'},
  {'coordinates': {'height': 91, 'width': 116, 'x': 76, 'y': 118},
   'label': 'apple'},
  {'coordinates': {'height': 103, 'width': 114, 'x': 75, 'y': 111},
   'label': 'apple'}]}

Visualize the bounding boxes on top of the training data:

In [ ]:
train_data['image_with_ground_truth'] = tc.object_detector.util.draw_bounding_boxes(
                                            train_data['image'], train_data['annotations'])
train_data.explore()

Train the model. This first downloads the Darknet feature extractor.

In [ ]:
model = tc.object_detector.create(train_data, feature='image', annotations='annotations')

Download completed: /var/folders/g2/d875dk7s4xq8j03j59rzx0w40000gn/T/model_cache/darknet.params
Download completed: /var/folders/g2/d875dk7s4xq8j03j59rzx0w40000gn/T/model_cache/darknet.mlmodel


Using GPU ("Intel(R) Iris(TM) Plus Graphics") to create model.

Setting 'batch_size' to 16

Setting 'max_iterations' to 26000

+--------------+--------------+--------------+

...

| 565          | 9.30433      | 27m 31s      |

| 570          | 9.30376      | 27m 56s      |

| 575          | 9.15788      | 28m 13s      |

| 580          | 9.27625      | 28m 38s      |

| 585          | 9.18657      | 28m 56s      |

Save the model. Also export to Core ML.

In [ ]:
model.save("SnackDetector.model")

In [ ]:
model.export_coreml("SnackDetector.mlmodel")

Load the model and evaluate it on the test set.

In [ ]:
model = tc.load_model("SnackDetector.model")

In [ ]:
val_data = load_images_with_annotations(val_dir, data_dir + "/annotations-val.csv")
test_data = load_images_with_annotations(test_dir, data_dir + "/annotations-test.csv")

In [ ]:
scores = model.evaluate(test_data)

`model.evaluate()` computes the "average precision" for each class, as well as the overall mean average precision metric. Higher is better.

In [ ]:
scores

Make predictions on the test data. This outputs something like this:

```
[{'confidence': 0.7225357099539148,
  'coordinates': {'height': 73.92794444010806,
                  'width': 90.45315889211807,
                  'x': 262.2198759929745,
                  'y': 155.496952970812},
  'label': 'dog',
  'type': 'rectangle'},
 ...]
```

which is similar to the annotations, but now there is a `confidence` field as well.

In [ ]:
test_data["predictions"] = model.predict(test_data)

In [ ]:
test_data.head()

Visualize the predicted bounding boxes on top of the test set:

In [ ]:
test_data['image_with_predictions'] = tc.object_detector.util.draw_bounding_boxes(
                                           test_data['image'], test_data['predictions'])

In [ ]:
test_data.head()

In [ ]:
test_data.explore()